In [1]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
import torchvision.models as models

# import statistics
import zipfile

import numpy as np
import random

import cv2
import os

# from google.colab import files, auth, drive
# drive.mount('/content/drive')
# drive.mount('/content/gdrive')

In [2]:
DEBUG = True

# GETTING THE DATA IN "EASY-TO-WORK-WITH" FORMAT !!!
kwargs = {}


class Args:

    def __init__(self):
        self.batch_size = 128
        self.epochs = 31
        self.lr = 0.00004
        self.sharpen_factor = 0.8   # reduce the learning rate by selected factor
        self.sharpen_frequency = 2  # every _ epoches
        self.seed = 1


args = Args()

In [3]:
mean, std = 42.71899717948718, 47.51068434986272
# MEANS = []
# STDEVS = []

use_cuda = torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")

if DEBUG:
    print(f"use_cuda = {use_cuda}")
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

use_cuda = True


In [4]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [5]:
class Organizer:

    def __init__(self, text_file, has_labels=True):
        self.link_dict = {}

        f = open(text_file, "r")
        content = f.read()

        rows = content.split("\n")

        for row in rows:
            if has_labels:
                values = row.split(",")
            else:
                values = [row, 0]
            if len(values) == 2:
                self.link_dict[values[0]] = values[1]

    def load_images(self, folder, rotation=False):  #, img_resize=30):
        images = []
        labels = []
        for filename in os.listdir(folder):
            img = cv2.imread(os.path.join(folder, filename), 0)
            dim = (100, 100)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            if rotation:
                img = rotate_image(img, random.randint(-20, 20))
            # mean, std = cv2.meanStdDev(img)
            # MEANS.append(float(mean))
            # STDEVS.append(float(std))
            img = normalize(to_tensor(img), [mean], [std])
            if img is not None:
                images.append(img)
                labels.append(int(self.link_dict[filename]))
        return torch.unsqueeze(torch.cat(images, dim=0), dim=1), torch.Tensor(labels)

    def submit(self, filename, labels):
        f = open(filename, "w")
        output_content = "id,label\n"
        for idx, key in enumerate(self.link_dict.keys()):
            if idx < len(labels):
                output_content += key
                output_content += ","
                output_content += str(labels[idx])
                output_content += "\n"
        f.write(output_content)

In [6]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):

        return self.x[idx], self.y[idx]

In [7]:
print(os.listdir("../input/ml-project/data"))

['validation', 'test.txt', 'validation.txt', 'train.txt', 'sample_submission.txt', 'test', 'train']


In [8]:
if DEBUG:
    print("Incepem incarcarea imaginilor de train")
training_data = Organizer("../input/ml-project/data/train.txt")

training_sets = []

print("Incarcam dataset-ul original de train")
X_train, y_train = training_data.load_images("../input/ml-project/data/train/", rotation=False)
original_training_set = MyDataset(X_train, y_train)

variations = 12

print(f"Done ! Incarcam {variations} variante usor modificate ale datasetului")

print("0.0%", end=" ")
for i in range(0, variations):
    X_train, y_train = training_data.load_images("../input/ml-project/data/train/", rotation=True)
    training_set = MyDataset(X_train, y_train)
    training_sets.append(training_set)
    print("{:.1f}%".format(((i + 1) / variations) * 100), end=" ")
print("")

Incepem incarcarea imaginilor de train
Incarcam dataset-ul original de train
Done ! Incarcam 12 variante usor modificate ale datasetului
0.0% 8.3% 16.7% 25.0% 33.3% 41.7% 50.0% 58.3% 66.7% 75.0% 83.3% 91.7% 100.0% 


In [9]:
if DEBUG:
    print("Done. Incepem incarcarea imaginilor de validare")
validation_data = Organizer("../input/ml-project/data/validation.txt")
X_validation, y_validation = validation_data.load_images("../input/ml-project/data/validation/")

if DEBUG:
    print("Done. Incepem incarcarea imaginilor de test")
testing_data = Organizer("../input/ml-project/data/test.txt", has_labels=False)
X_test, zeroes = testing_data.load_images("../input/ml-project/data/test/")
if DEBUG:
    print("Done")

Done. Incepem incarcarea imaginilor de validare
Done. Incepem incarcarea imaginilor de test
Done


In [10]:
validation_set = MyDataset(X_validation, y_validation)

testing_set = MyDataset(X_test, zeroes)

In [11]:
train_loaders = []

no_of_train_loaders = 0

if DEBUG:
    print("Facem data loaderele")
original_train_loader = DataLoader(dataset=original_training_set, batch_size=args.batch_size, shuffle=False, **kwargs)
for i in range(0, variations):
    train_loader = DataLoader(dataset=training_sets[i], batch_size=args.batch_size, shuffle=False, **kwargs)
    train_loaders.append(train_loader)
    no_of_train_loaders += 1
    if i % 4 == 0:
        train_loaders.append(original_train_loader)
        no_of_train_loaders += 1

Facem data loaderele


In [12]:
valid_loader = DataLoader(dataset=validation_set, batch_size=1, shuffle=False, **kwargs)

test_loader = DataLoader(dataset=testing_set, batch_size=1, shuffle=False, **kwargs)
if DEBUG:
    print("Done")

Done


In [13]:
channels_conv01 = 512
channels_conv02 = 512
channels_conv03 = 512
channels_conv04 = 256
channels_conv05 = 256
channels_conv06 = 160
channels_conv07 = 192
channels_conv08 = 256
channels_conv09 = 320
channels_conv10 = 512
connection = 6400
features_fc1 = 1024
features_fc2 = 1024
features_fc3 = 1024
features_fc4 = 768
dropout_features = 0.1
dropout_weights = 0.15


class Network(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer01 = nn.Conv2d(in_channels=3, out_channels=channels_conv01, kernel_size=(7, 7), stride=(2, 2))
        self.layer02 = nn.Conv2d(in_channels=channels_conv01, out_channels=channels_conv02, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
        self.layer03 = nn.Conv2d(in_channels=channels_conv02, out_channels=channels_conv03, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
        self.layer04 = nn.Conv2d(in_channels=channels_conv03, out_channels=channels_conv04, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer05 = nn.Conv2d(in_channels=channels_conv04, out_channels=channels_conv05, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

        self.linear1 = nn.Linear(connection, features_fc1)
        self.linear2 = nn.Linear(features_fc1, features_fc2)
        self.linear3 = nn.Linear(features_fc2, features_fc3)
        self.linear4 = nn.Linear(features_fc3, features_fc4)
        self.linear5 = nn.Linear(features_fc4, 3)

        self.norm01 = nn.BatchNorm2d(channels_conv01)
        self.norm02 = nn.BatchNorm2d(channels_conv02)
        self.norm03 = nn.BatchNorm2d(channels_conv03)
        self.norm04 = nn.BatchNorm2d(channels_conv04)
        self.norm05 = nn.BatchNorm2d(channels_conv05)

        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)
        self.maxPool = nn.MaxPool2d(2, 2)
        self.dropout_features = nn.Dropout(dropout_features)
        self.dropout_weights = nn.Dropout(dropout_weights)

    def forward(self, output):
        output = F.relu(self.norm01(self.layer01(output)))
        output = self.dropout_features(output)
        output = F.relu(self.norm02(self.maxPool(self.layer02(output))))
        output = self.dropout_features(output)
        output = F.relu(self.norm03(self.layer03(output)))
        output = self.dropout_features(output)
        output = F.relu(self.norm04(self.maxPool(self.layer04(output))))
        output = self.dropout_features(output)
        output = F.relu(self.norm05(self.maxPool(self.layer05(output))))
        output = self.dropout_features(output)

        output = self.flatten(output)
        # print(output.shape)
        output = self.linear1(output)
        output = self.dropout_weights(output)
        output = self.linear2(output)
        output = self.dropout_weights(output)
        output = self.linear3(output)
        output = self.dropout_weights(output)
        output = self.linear4(output)
        output = self.dropout_weights(output)
        output = self.linear5(output)
        output = self.softmax(output)

        return output

In [14]:
# Definirea retelei
# network = Network().to(device)
choosen =  models.resnet.resnext101_32x8d(pretrained=True)
"""
# VGG16
choosen.classifier.add_module("7", nn.ReLU(inplace=True))
choosen.classifier.add_module("8", nn.Dropout(p=0.5, inplace=False))
choosen.classifier.add_module("9", nn.Linear(in_features=1000, out_features=3, bias=True))
"""
"""
# resnet18
choosen.add_module("fc_2", nn.Linear(in_features=1000, out_features=768, bias=True))
choosen.add_module("fc_3", nn.Linear(in_features=768, out_features=3, bias=True))
"""
"""
# VGG19
choosen.classifier.add_module("7", nn.ReLU(inplace=True))
choosen.classifier.add_module("8", nn.Dropout(p=0.3, inplace=False))
choosen.classifier.add_module("9", nn.Linear(in_features=1000, out_features=768, bias=True))
choosen.classifier.add_module("10", nn.ReLU(inplace=True))
choosen.classifier.add_module("11", nn.Dropout(p=0.3, inplace=False))
choosen.classifier.add_module("12", nn.Linear(in_features=768, out_features=3, bias=True))
"""

# resnet
choosen.add_module("fcD", nn.Dropout(p=0.2, inplace=False))
choosen.add_module("fc_2", nn.Linear(in_features=1000, out_features=768, bias=True))
choosen.add_module("fc_2D", nn.Dropout(p=0.2, inplace=False))
choosen.add_module("fc_3", nn.Linear(in_features=768, out_features=768, bias=True))
choosen.add_module("fc_3D", nn.Dropout(p=0.2, inplace=False))
choosen.add_module("fc_4", nn.Linear(in_features=768, out_features=3, bias=True))

"""
# densenet
choosen.add_module("classifier2", nn.ReLU(inplace=True))
choosen.add_module("classifier3", nn.Dropout(p=0.2, inplace=False))
choosen.add_module("classifier4", nn.Linear(in_features=1000, out_features=768, bias=True))
choosen.add_module("classifier5", nn.ReLU(inplace=True))
choosen.add_module("classifier6", nn.Dropout(p=0.2, inplace=False))
choosen.add_module("classifier7", nn.Linear(in_features=768, out_features=768, bias=True))
choosen.add_module("classifier8", nn.ReLU(inplace=True))
choosen.add_module("classifier9", nn.Dropout(p=0.2, inplace=False))
choosen.add_module("classifier10", nn.Linear(in_features=768, out_features=3, bias=True))
"""
choosen = choosen.to(device)
network = choosen
print(network)

# Definirea Optimizatorului
optimizer = optim.Adam(network.parameters(), lr=args.lr)
optimizer.zero_grad()

# Definirea functiei de cost pentru clasificare
loss_fn = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


  0%|          | 0.00/340M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [15]:
def train_fn(epochs: int, train_loaders, validation_loader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):

    max_acc = 0
    if DEBUG:
        print("Begin training...")
    for epoch in range(epochs):
        if epoch % args.sharpen_frequency == 0:
            args.lr = args.lr * args.sharpen_factor
            optimizer = optim.Adam(network.parameters(), lr=args.lr)
        # model.dropout_features = nn.Dropout(dropout_features)
        # model.dropout_weights = nn.Dropout(dropout_weights)
    # Iteram prin fiecare exemplu din dataset
        for images, labels in train_loaders[(epoch % no_of_train_loaders)]:
            images = images.repeat(1, 3, 1, 1)
            images = images.to(device)
            labels = labels.to(device).long()
          # Aplicam reteaua neurala pe imaginile de intrare
            out = model(images)
          # Aplicam functia cost pe iesirea retelei neurale si pe adnotarile imaginilor
            loss = loss_fn(out, labels)
          # Aplicam algoritmul de back-propagation
            loss.backward()
          # Facem pasul de optimizare, pentru a aplica gradientii pe parametrii retelei
            optimizer.step()
          # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
            optimizer.zero_grad()

        print(f"Loss-ul dupa epoca {epoch + 1} este {loss.item()}")

         # Caluculul acuratetii
        # model.dropout_features = nn.Dropout(0)
        # model.dropout_weights = nn.Dropout(0)
        count = len(validation_loader)
        correct = 0
        model.eval()
        with torch.no_grad():
            for val_image, val_label in validation_loader:
                val_image = val_image.repeat(1, 3, 1, 1)
                if use_cuda:
                    val_label = val_label.cuda()
                    val_image = val_image.cuda()
                    out_class = torch.argmax(model(val_image))
                if out_class == val_label:
                    correct += 1

        current_acc = (correct / count) * 100
        print(f"Acuratetea dupa epoca {epoch + 1} este {current_acc}%")
        if current_acc > max_acc:
            max_acc = current_acc
            torch.save(model.state_dict(), "model.pth")
            print("Starea modelului s-a salvat ! Acuratete estimata: {:.2f}%".format(current_acc))
    if DEBUG:
        print(f"Max accuracy obtained: {max_acc}")

In [16]:
def predict(model: nn.Module):

    f = open("../input/ml-project/data/test.txt", "r")
    content = f.read()

    img_names = content.split("\n")

    images = []

    for name in img_names:
        if name != "":
            img = cv2.imread(os.path.join("../input/ml-project/data/test/", name), 0)
            dim = (100, 100)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            img = normalize(to_tensor(img), [mean], [std])
            img = torch.unsqueeze(img, dim=0)
            images.append(img)

    # model.dropout_features = nn.Dropout(0)
    # model.dropout_weights = nn.Dropout(0)
    
    f = open("submision.txt", "w")
    output_content = "id,label\n"
    
    predictions = []
    model.eval()
    with torch.no_grad():
        for idx, image in enumerate(images):
            image = image.repeat(1, 3, 1, 1)
            image = image.to(device)
            label = torch.argmax(model(image))
            label = label.detach().cpu().numpy()
            
            output_content += img_names[idx]
            output_content += ","
            output_content += str(label)
            output_content += "\n"
            
            predictions.append(label)

    f.write(output_content)

    return predictions

In [17]:
# print(f"Mean: {statistics.mean(MEANS)}; STD: {statistics.mean(STDEVS)}")
train_fn(args.epochs, train_loaders, valid_loader, network, loss_fn, optimizer)

Begin training...
Loss-ul dupa epoca 1 este 0.7833027243614197
Acuratetea dupa epoca 1 este 66.95555555555556%
Starea modelului s-a salvat ! Acuratete estimata: 66.96%
Loss-ul dupa epoca 2 este 1.1056023836135864
Acuratetea dupa epoca 2 este 37.75555555555555%
Loss-ul dupa epoca 3 este 0.7060490250587463
Acuratetea dupa epoca 3 este 61.8%
Loss-ul dupa epoca 4 este 0.4534675180912018
Acuratetea dupa epoca 4 este 63.11111111111111%
Loss-ul dupa epoca 5 este 0.22940777242183685
Acuratetea dupa epoca 5 este 70.06666666666666%
Starea modelului s-a salvat ! Acuratete estimata: 70.07%
Loss-ul dupa epoca 6 este 0.22463756799697876
Acuratetea dupa epoca 6 este 69.24444444444444%
Loss-ul dupa epoca 7 este 0.1661151647567749
Acuratetea dupa epoca 7 este 71.33333333333334%
Starea modelului s-a salvat ! Acuratete estimata: 71.33%
Loss-ul dupa epoca 8 este 0.19777363538742065
Acuratetea dupa epoca 8 este 73.33333333333333%
Starea modelului s-a salvat ! Acuratete estimata: 73.33%
Loss-ul dupa epoca 9

In [18]:
print(os.listdir("../input/ml-project/data"))
print(os.listdir("../input/modelkagle"))

['validation', 'test.txt', 'validation.txt', 'train.txt', 'sample_submission.txt', 'test', 'train']
['model_kaggle.pth']


In [19]:
# my_model = models.alexnet(pretrained=True)
# my_model.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
# my_model = my_model.to(device)

# my_model = Network().to(device)

my_model = choosen.to(device)
# my_model.load_state_dict(torch.load("../input/modelkagle/model_kaggle.pth"))
my_model.load_state_dict(torch.load("model.pth"))

predict(my_model)

# testing_data.submit("submision.txt", preds)

if DEBUG:
    print("Submisions successfully saved in submisions.txt !")

Submisions successfully saved in submisions.txt !


The end...